In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkConf

In [2]:
spark_conf = SparkConf()
spark_conf.setMaster("spark://master:7077")
spark_conf.setAppName("Lab6_Exercises")
spark_conf.set("spark.driver.memory", "2g")
spark_conf.set("spark.executor.cores", "1")
spark_conf.set("spark.driver.cores", "1")

# create the spark session, which is the entry point to Spark SQL engine.
spark = SparkSession.builder.config(conf=spark_conf).getOrCreate()

### Exercise 1

In [3]:
sales_df = spark.read.format("csv").option("header", "true").load("../data/sales_e1.csv")
sales_df.printSchema()
sales_df.show(5)

root
 |-- order_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- date: string (nullable = true)
 |-- quantity: string (nullable = true)

+--------+----------+----------+--------+
|order_id|product_id|      date|quantity|
+--------+----------+----------+--------+
|       1|        31|2021-08-10|      92|
|       2|        38|2021-08-02|      46|
|       3|        47|2021-08-01|      48|
|       4|        33|2021-08-09|      18|
|       5|        29|2021-08-05|      39|
+--------+----------+----------+--------+
only showing top 5 rows



In [4]:
products_df = spark.read.format("csv").option("header", "true").load("../data/products_e1.csv")
products_df.printSchema()
products_df.show(5)

root
 |-- product_id: string (nullable = true)
 |-- product_name: string (nullable = true)
 |-- unit_price: string (nullable = true)

+----------+------------+----------+
|product_id|product_name|unit_price|
+----------+------------+----------+
|         0|   product_0|        22|
|         1|   product_1|         2|
|         2|   product_2|         6|
|         3|   product_3|         3|
|         4|   product_4|        12|
+----------+------------+----------+
only showing top 5 rows



**Exercise 1.1**

Find the best performing product in terms of the total price of the sold items for any date

In [5]:
from pyspark.sql.functions import *
from pyspark.sql import Row, Window

In [6]:
# Calcuate the number of pieces sold by each seller for each product
sales_df_total = sales_df.groupby(col("product_id")).agg(sum("quantity").alias("total_quantity"))

In [7]:
# https://kb.databricks.com/data/join-two-dataframes-duplicated-columns.html
# https://stackoverflow.com/questions/35258506/how-to-avoid-duplicate-columns-after-join
join_expression = ["product_id"]

merged_df = sales_df_total.join(products_df, join_expression,"left").withColumn("total_price", col("total_quantity") * col("unit_price"))
merged_df.show(10)

print(merged_df.orderBy(col("total_price").desc()).select("product_name", "total_price").collect()[0])

+----------+--------------+------------+----------+-----------+
|product_id|total_quantity|product_name|unit_price|total_price|
+----------+--------------+------------+----------+-----------+
|         7|        1924.0|   product_7|        15|    28860.0|
|        15|        3075.0|  product_15|        15|    46125.0|
|        11|        2422.0|  product_11|         4|     9688.0|
|        29|        2868.0|  product_29|         1|     2868.0|
|        42|        2414.0|  product_42|         3|     7242.0|
|         3|        1859.0|   product_3|         3|     5577.0|
|        30|        2175.0|  product_30|        12|    26100.0|
|        34|        3582.0|  product_34|         9|    32238.0|
|         8|        2775.0|   product_8|         5|    13875.0|
|        28|        2270.0|  product_28|         5|    11350.0|
+----------+--------------+------------+----------+-----------+
only showing top 10 rows

Row(product_name='product_19', total_price=47264.0)


**Exercise 1.2**

Find the best preforming product and the worst performing product in terms of the total price of the sold items for each date.

In [8]:
sales_df_d = sales_df.select("*", col("date"), to_date(col("date"),"yyyy-MM-dd").alias("sdate")).drop("date")

# Calcuate the number of pieces sold by each seller for each product on each date
sales_df_d_total = sales_df_d.groupby(col("product_id"), col("sdate")).agg(sum("quantity").alias("total_quantity"))

join_expression = ["product_id"] # as the two data frames use the same column id
sales_df_d_total_price = sales_df_d_total.join(products_df, join_expression,"left").withColumn("total_price", col("total_quantity") * col("unit_price"))

# Rank the product in terms of the total price, per each date. Then, select the best and worst product using ranks
window_desc = Window.partitionBy(col("sdate")).orderBy(col("total_price").desc())
window_asc = Window.partitionBy(col("sdate")).orderBy(col("total_price").asc())

sales_df_d_total_price.printSchema()
sales_df_d_total_price.show(4)

sales_df_d_total_windowed = sales_df_d_total_price.withColumn("rank_desc", dense_rank().over(window_desc)).withColumn("rank_asc", dense_rank().over(window_asc))

# Get the best and worst performing products
sales_df_d_total_windowed.where((col("rank_desc") == 1) | (col("rank_asc") == 1) ).select("*").show(100)

root
 |-- product_id: string (nullable = true)
 |-- sdate: date (nullable = true)
 |-- total_quantity: double (nullable = true)
 |-- product_name: string (nullable = true)
 |-- unit_price: string (nullable = true)
 |-- total_price: double (nullable = true)

+----------+----------+--------------+------------+----------+-----------+
|product_id|     sdate|total_quantity|product_name|unit_price|total_price|
+----------+----------+--------------+------------+----------+-----------+
|         5|2021-08-05|          95.0|   product_5|        10|      950.0|
|         2|2021-08-05|         242.0|   product_2|         6|     1452.0|
|        33|2021-08-04|         419.0|  product_33|         9|     3771.0|
|        10|2021-08-11|         106.0|  product_10|        10|     1060.0|
+----------+----------+--------------+------------+----------+-----------+
only showing top 4 rows

+----------+----------+--------------+------------+----------+-----------+---------+--------+
|product_id|     sdate|

### Exercise 2

In [9]:
# Setup hadoop fs configuration for schema gs://
conf = spark.sparkContext._jsc.hadoopConfiguration()
conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
conf.set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")

In [10]:
#  Google Storage File Path
gsc_file_path = "gs://delabs-lab6-input/restaurants_geo.csv"

# Create data frame
restaurants_geo_df = spark.read.format("csv").option("header", "true").option("delimiter", ";").load(gsc_file_path)
restaurants_geo_df.printSchema()
restaurants_geo_df.show(5)

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- address: string (nullable = true)
 |-- city: string (nullable = true)
 |-- phone: string (nullable = true)
 |-- type: string (nullable = true)

+---+--------------------+--------------------+-----------+------------+-----------+
| id|                name|             address|       city|       phone|       type|
+---+--------------------+--------------------+-----------+------------+-----------+
|  1|arnie morton's of...|435 s. la cienega...|los angeles|310/246-1501|   american|
|  2|arnie morton's of...|435 s. la cienega...|los angeles|310-246-1501|steakhouses|
|  3|  art's delicatessen| 12224 ventura blvd.|studio city|818/762-1221|   american|
|  4|          art's deli| 12224 ventura blvd.|studio city|818-762-1221|      delis|
|  5|       hotel bel-air|701 stone canyon rd.|    bel air|310/472-1211|californian|
+---+--------------------+--------------------+-----------+------------+-----------+
only showin

In [11]:
restaurants_df = spark.read.format("bigquery").load("delabs.lab6.restaurants")
restaurants_df.printSchema()
restaurants_df.show(4)

root
 |-- rid: long (nullable = true)
 |-- avg_price: double (nullable = true)
 |-- rating: double (nullable = true)
 |-- contact: string (nullable = true)

+---+---------+------+-------------------+
|rid|avg_price|rating|            contact|
+---+---------+------+-------------------+
|167|    37.64|   2.0|sdarragh4m@yale.edu|
|602|    54.41|   2.0|   abatistegp@51.la|
|289|    52.18|  2.25|  cceely80@hibu.com|
|338|    98.83|  2.25|mmcerlaine9d@w3.org|
+---+---------+------+-------------------+
only showing top 4 rows



**Exercise 2.1**

Give me the contact details of the cheapest restaurant and the most expensive restaurant in each city of USA.

In [12]:
from pyspark.sql.functions import *
from pyspark.sql import Row, Window

In [13]:
join_expression = restaurants_geo_df["id"] == restaurants_df["rid"]

In [14]:
# default join is the inner or left join
restaurants_merged = restaurants_geo_df.join(restaurants_df, join_expression,"left").drop("rid")
restaurants_merged.show(10)

+---+--------------------+--------------------+-----------+------------+-----------+---------+------+--------------------+
| id|                name|             address|       city|       phone|       type|avg_price|rating|             contact|
+---+--------------------+--------------------+-----------+------------+-----------+---------+------+--------------------+
|  1|arnie morton's of...|435 s. la cienega...|los angeles|310/246-1501|   american|    58.98|  2.49|lbuckeridge0@indi...|
|  1|arnie morton's of...|435 s. la cienega...|los angeles|310/246-1501|   american|    58.98|  2.49|lbuckeridge0@indi...|
|  1|arnie morton's of...|435 s. la cienega...|los angeles|310/246-1501|   american|    58.98|  2.49|lbuckeridge0@indi...|
|  1|arnie morton's of...|435 s. la cienega...|los angeles|310/246-1501|   american|    58.98|  2.49|lbuckeridge0@indi...|
|  2|arnie morton's of...|435 s. la cienega...|los angeles|310-246-1501|steakhouses|    60.32|  3.69|wstrickland1@goog...|
|  2|arnie morto

In [15]:
# Rank the resturants in terms of the avg price, per each city. Then, select the best and worst product using ranks
window_desc = Window.partitionBy(col("city")).orderBy(col("avg_price").desc())
window_asc = Window.partitionBy(col("city")).orderBy(col("avg_price").asc())

In [16]:
restaurants_merged_windowed = restaurants_merged.withColumn("rank_desc", dense_rank().over(window_desc)).withColumn("rank_asc", dense_rank().over(window_asc))

In [17]:
# Get the best and worst performing products
cheap_expensive_city_table1 = restaurants_merged_windowed.where((col("rank_desc") == 1) | (col("rank_asc") == 1) ).select("*")
cheap_expensive_city_table1.show(10)

cheap_expensive_city_table2 = cheap_expensive_city_table1.withColumn("price_category", when(col("rank_desc") == 1, "Most Expensive").otherwise("Cheapest")).select(
    "city",
    "price_category",
    "avg_price",
    "name",
    "contact",
    "phone",
)
cheap_expensive_city_table2.show(10)

+---+-------------------+-------------------+-----------+------------+-------------+---------+------+--------------------+---------+--------+
| id|               name|            address|       city|       phone|         type|avg_price|rating|             contact|rank_desc|rank_asc|
+---+-------------------+-------------------+-----------+------------+-------------+---------+------+--------------------+---------+--------+
|673|      john o'groats|10516 w. pico blvd.|    west la|310-204-0692| coffee shops|    56.01|  3.33|jdumberellio@post...|        4|       1|
|673|      john o'groats|10516 w. pico blvd.|    west la|310-204-0692| coffee shops|    56.01|  3.33|jdumberellio@post...|        4|       1|
|673|      john o'groats|10516 w. pico blvd.|    west la|310-204-0692| coffee shops|    56.01|  3.33|jdumberellio@post...|        4|       1|
|673|      john o'groats|10516 w. pico blvd.|    west la|310-204-0692| coffee shops|    56.01|  3.33|jdumberellio@post...|        4|       1|
|665|f

In [18]:
# Use the Cloud Storage bucket for temporary BigQuery export data used by the connector.
bucket = "delabs-lab6-temp"
spark.conf.set("temporaryGcsBucket", bucket)

# Saving the data to BigQuery
cheap_expensive_city_table2.write.format("bigquery").option("table", "delabs.lab6.resturants_selection").save()

In [19]:
# Stop the spark context
spark.stop()